<img src='img/fon.png' width=200>
<center>
    <br/>
    Otkrivanje zakonitosti u podacima 2019-2020
    <br/>
    Fakultet Organizacionih Nauka, 2020
    <br/>
    <h1>D2. Domaći zadatak 2</h1>
</center>

# Zahtevi domaćeg zadatka


0. Učitati podatke o vinima (wine_quality.csv), sa ciljem prepoznavanja "top_quality" klase.

1. Kreirati minimalno 3 modela za klasifikaciju (sa default parametrima)
    - proceniti grešku generalizacije za sve modele, na osnovu nekoliko metrika;
    - prikazati i standardnu devijaciju procene greške (od 10 kros-validacionih uzoraka)
    - odgovoriti da li su modeli pretrenirani;
    - prodiskutovati koja metrika je najadekvatnija za ovaj problem, i koji tip grešaka je skuplji;
    - modifikovati granicu odlučivanja, tako da model pravi više jeftinijih grešaka;
    - odgovoriti koji atributi su najbitniji za naučene modele (gde je moguće videti)

2. Optimizovati minimalno 2 parametra kod najboljeg modela (iz koraka 1)
    - ispitati najmanje 100 različitih vrednosti parametara
    - uporediti kvalitet optimizovanog i neoptimizovanog modela

3. Napraviti 2 razlicita ansambl modela, bazirana na modelima iz koraka 2
    - optimizovati parametre ansambl modela
    - uporediti kvalitet ansambl i osnovnih modela

In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## Učitavanje podataka

0. Učitati podatke o vinima (wine_quality.csv), sa ciljem prepoznavanja "top_quality" klase.

In [2]:
df = pd.read_csv('data/wine_quality.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,top_quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0


In [3]:
df.shape

(4898, 12)

## Uvidi o potrebnom preprocesiranju

Svi atributi su numerički, tako da nema potrebe za enkodovanjem. Na različitim su skalama, pa ih moramo normalizovati. Vrednosti target klase su u odnosu 4:1, pa bi bilo dobro balansirati podatke, što će biti prikazano nakon ispunjenja obaveznih zahteva.

## Kreiranje modela

### Otklanjanje izuzetaka

In [4]:
from scipy import stats

df_no_out = df.loc[(np.abs(stats.zscore(df)) < 3).all(axis=1), :]

In [5]:
print(f'{str(df.shape[0] - df_no_out.shape[0])} izuzetaka je izbačeno. Procenat dataseta: {str((df.shape[0] - df_no_out.shape[0]) / df.shape[0])}')

396 izuzetaka je izbačeno. Procenat dataseta: 0.08084932625561454


### Podela na trening i test podatke

In [6]:
X = df_no_out.drop('top_quality', axis=1)
y = df_no_out['top_quality']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [8]:
print(f'Velicina treninga: {str(X_train.shape[0])}. Velicina testa: {str(X_test.shape[0])}')

Velicina treninga: 3151. Velicina testa: 1351


### Skaliranje podataka

Korišćen je MaxMinScaler koji je "naučen" na treningu, a onda samo iskorišćen za transformaciju testa.

In [9]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_train

array([[0.79591837, 0.38      , 0.49275362, ..., 0.45882353, 0.47540984,
        0.32758621],
       [0.28571429, 0.46      , 0.47826087, ..., 0.45882353, 0.62295082,
        0.36206897],
       [0.79591837, 0.14      , 0.56521739, ..., 0.34117647, 0.27868852,
        0.36206897],
       ...,
       [0.51020408, 0.34      , 0.42028986, ..., 0.25882353, 0.2295082 ,
        0.56896552],
       [0.42857143, 0.6       , 0.49275362, ..., 0.41176471, 0.8852459 ,
        0.27586207],
       [0.32653061, 0.2       , 0.44927536, ..., 0.64705882, 0.59016393,
        0.46551724]])

### Inicijalni modeli

1. Kreirati minimalno 3 modela za klasifikaciju (sa default parametrima)

In [10]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
from sklearn.tree import DecisionTreeClassifier

model_tree = DecisionTreeClassifier()
model_tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [12]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier()
model_knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [13]:
from sklearn.ensemble import RandomForestClassifier

model_rff = RandomForestClassifier()
model_rff.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Evaluacija 

    - proceniti grešku generalizacije za sve modele, na osnovu nekoliko metrika;

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score
from prettytable import PrettyTable

In [15]:
def evaluate(models, scores, X=X_train, y=y_train):
    t = PrettyTable(['Model\Mera evaluacije'] + [str(score).split()[1] for score in scores])
    for model in models:
        y_pred = model.predict(X)
        t.add_row([str(model).split('(')[0]] + [score(y, y_pred) for score in scores])
    print(t)
    print('\n')

In [16]:
models = [model_lr, model_tree, model_knn, model_rff]
scores = [accuracy_score, precision_score, recall_score, roc_auc_score]

evaluate(models, scores, X_train, y_train)

+------------------------+--------------------+--------------------+---------------------+--------------------+
| Model\Mera evaluacije  |   accuracy_score   |  precision_score   |     recall_score    |   roc_auc_score    |
+------------------------+--------------------+--------------------+---------------------+--------------------+
|   LogisticRegression   | 0.7959377975245954 | 0.6161290322580645 | 0.26713286713286716 | 0.6091411462101118 |
| DecisionTreeClassifier |        1.0         |        1.0         |         1.0         |        1.0         |
|  KNeighborsClassifier  | 0.8803554427165979 | 0.7835570469798657 |  0.6531468531468532 | 0.8000955940611113 |
| RandomForestClassifier | 0.9879403364011425 | 0.9985272459499264 |  0.9482517482517483 | 0.9739206196102748 |
+------------------------+--------------------+--------------------+---------------------+--------------------+




In [17]:
evaluate(models, scores, X_test, y_test)

+------------------------+--------------------+--------------------+---------------------+--------------------+
| Model\Mera evaluacije  |   accuracy_score   |  precision_score   |     recall_score    |   roc_auc_score    |
+------------------------+--------------------+--------------------+---------------------+--------------------+
|   LogisticRegression   | 0.7868245743893413 | 0.5818181818181818 | 0.20915032679738563 | 0.5825655940207024 |
| DecisionTreeClassifier | 0.8179126572908957 |        0.6         |  0.5882352941176471 | 0.7367013791162398 |
|  KNeighborsClassifier  | 0.8164322723908216 | 0.6141732283464567 |  0.5098039215686274 | 0.7080120086312037 |
| RandomForestClassifier | 0.8638045891931903 | 0.7877358490566038 |  0.545751633986928  | 0.751344716514995  |
+------------------------+--------------------+--------------------+---------------------+--------------------+




#### Evaluacija unakrsnom kros-validacijom

    - prikazati i standardnu devijaciju procene greške (od 10 kros-validacionih uzoraka)

In [18]:
from sklearn.model_selection import cross_val_score

for model, label in zip([model_lr, model_tree, model_knn, model_rff], ['Logisticka regresija', 'Stablo odlucivanja', 'K-NN', 'Random Forest']):
    cv_scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')
    print('[%s] Tacnost: %0.2f +/- %0.2f' % (label, cv_scores.mean(), cv_scores.std()))

[Logisticka regresija] Tacnost: 0.78 +/- 0.03
[Stablo odlucivanja] Tacnost: 0.74 +/- 0.04
[K-NN] Tacnost: 0.74 +/- 0.03
[Random Forest] Tacnost: 0.80 +/- 0.02


    - odgovoriti da li su modeli pretrenirani;

Modeli jesu pretrenirani jer postoji varijacija od 2-4% u tačnosti. Posebno je pretreniran DT model. što je očekivano jer nije ograničen nijednim meta-parametrom. Takođe se još iz standardne trening-test evaluacije jasno vidi da su modeli KNN i RF takođe peretrenirani, jer razlike u tačnosti na ova dva skupa su oko 15%.

    - prodiskutovati koja metrika je najadekvatnija za ovaj problem, i koji tip grešaka je skuplji;
    
Naša pozitivna klasa je da je vino dobrog kvaliteta. Ako na problem gledamo kao da želimo da otkrijemo sva vina koja su dobra, onda nas greška da ne otkrijemo neko dobro vino (FN) mnogo više košta nego da za neko loše vino kažemo da je dobro (FP). Ako propustimo dobro vino ono ostaje neotkriveno, dok ako za neko loše kažemo da je dobro, u najgorem slučaju, ćemo ga probati i zaključiti da je ipak loše. Dakle, adekvatna mera evaulacije je ona koja naglašava značaj FN greške - to je odziv (Recall).

![precision](img/precision.png)

#### Modifikovanje praga odlučivanja

    - modifikovati granicu odlučivanja, tako da model pravi više jeftinijih grešaka;
    

In [19]:
def evaluate_with_treshold(models, scores, threshold, X, y):
    t = PrettyTable(['Model\Mera evaluacije'] + [str(score).split()[1] for score in scores])
    for model in models:
        y_pred = (model.predict_proba(X)[:,1] >= threshold).astype(bool)
        t.add_row([str(model).split('(')[0]] + [score(y, y_pred) for score in scores])
    print(t)
    print('\n')

In [20]:
evaluate_with_treshold(models, scores, 0.3, X_train, y_train)

+------------------------+--------------------+--------------------+--------------------+--------------------+
| Model\Mera evaluacije  |   accuracy_score   |  precision_score   |    recall_score    |   roc_auc_score    |
+------------------------+--------------------+--------------------+--------------------+--------------------+
|   LogisticRegression   | 0.7638844811171057 | 0.4838709677419355 | 0.6083916083916084 | 0.7089577089577089 |
| DecisionTreeClassifier |        1.0         |        1.0         |        1.0         |        1.0         |
|  KNeighborsClassifier  | 0.8562361155188829 | 0.6347736625514403 | 0.862937062937063  | 0.8586031784307646 |
| RandomForestClassifier | 0.9679466835925103 | 0.8780788177339901 | 0.9972027972027973 | 0.9782812015570637 |
+------------------------+--------------------+--------------------+--------------------+--------------------+




Prikazaćemo rezultate početnih modela i modela sa pomerenom granicom odlučivanja kako bismo lakše poredili šta se desilo sa pomeranjem granice.

In [21]:
evaluate(models, scores, X_test, y_test)

+------------------------+--------------------+--------------------+---------------------+--------------------+
| Model\Mera evaluacije  |   accuracy_score   |  precision_score   |     recall_score    |   roc_auc_score    |
+------------------------+--------------------+--------------------+---------------------+--------------------+
|   LogisticRegression   | 0.7868245743893413 | 0.5818181818181818 | 0.20915032679738563 | 0.5825655940207024 |
| DecisionTreeClassifier | 0.8179126572908957 |        0.6         |  0.5882352941176471 | 0.7367013791162398 |
|  KNeighborsClassifier  | 0.8164322723908216 | 0.6141732283464567 |  0.5098039215686274 | 0.7080120086312037 |
| RandomForestClassifier | 0.8638045891931903 | 0.7877358490566038 |  0.545751633986928  | 0.751344716514995  |
+------------------------+--------------------+--------------------+---------------------+--------------------+




In [22]:
evaluate_with_treshold(models, scores, 0.3, X_test, y_test)

+------------------------+--------------------+---------------------+--------------------+--------------------+
| Model\Mera evaluacije  |   accuracy_score   |   precision_score   |    recall_score    |   roc_auc_score    |
+------------------------+--------------------+---------------------+--------------------+--------------------+
|   LogisticRegression   | 0.7572168763878608 | 0.47058823529411764 | 0.5751633986928104 | 0.6928448572411421 |
| DecisionTreeClassifier | 0.8179126572908957 |         0.6         | 0.5882352941176471 | 0.7367013791162398 |
|  KNeighborsClassifier  | 0.7668393782383419 | 0.48951048951048953 | 0.6862745098039216 | 0.7383525659067455 |
| RandomForestClassifier | 0.8134715025906736 |  0.5602678571428571 | 0.8202614379084967 | 0.8158723457485068 |
+------------------------+--------------------+---------------------+--------------------+--------------------+




Vidimo, dakle, da smo pomeranjem granice odlučivanja žrtvovali precision i accuracy kako bismo dobili bolji recall. Primetimo da smo, osim kod DT, kod svih ostalih povećali i roc_auc score. DT model nije promenio nijedan score, što znači da taj model daje prilično ekstremne verovatnoće, pa klase nisu promenjene ni prilikom pomeranja granice.
Kod ostalih modela, razlike su velike jer je prag odlučivanja spušten za čak 0.2. Ovakav prag nam odgovara samo u slučaju da nam je baš bitno da ne propustimo mnogo vina top kvaliteta, a da nam je jeftina FP greška, koju ćemo u ovom slučaju često praviti (primetiti _precision_). 

    - odgovoriti koji atributi su najbitniji za naučene modele (gde je moguće videti)

In [23]:
feature_importance = pd.DataFrame({'feature': df.columns[:-1], 
              'lr_coefs': model_lr.coef_[0],
              'rf_importance': model_rff.feature_importances_})

In [24]:
feature_importance.sort_values(by='rf_importance', ascending=False)

,feature,lr_coefs,rf_importance
10,alcohol,3.463778,0.181967
7,density,-2.687755,0.102824
5,free sulfur dioxide,0.885810,0.090180
8,pH,1.763527,0.090029
1,volatile acidity,-1.508382,0.085731
4,chlorides,-1.850007,0.085197
3,residual sugar,2.086232,0.081874
6,total sulfur dioxide,-0.289282,0.076215
0,fixed acidity,0.846896,0.069414
2,citric acid,-0.086910,0.068357


Iako potpuno drugačiji modeli, logistička i slučajne šume se slažu oko bitnosti većine atributa. Ono što možemo da primetimo je da se jedino `residual sugar` i `violatile acidity` pokazali kao bitniji u logističkoj, dok se sa tim šume ne slažu. Podsetimo da se LR model veoma slabo bori sa našim nebalansiranim podacima, tako da nam je bolje verovati RF modelu.
Sa druge strane, možemo primetiti da su nam koeficijenti logističke dosta korisniji u objašnjavanju modela jer tačno vidimo da li atribut utiče pozitivno ili negativno na ciljnu klasu.

### Optimizacija metaparametara

2. Optimizovati minimalno 2 parametra kod najboljeg modela (iz koraka 1)
    - ispitati najmanje 100 različitih vrednosti parametara
    - uporediti kvalitet optimizovanog i neoptimizovanog modela

Primećujemo da je prvi KNN model koji je napravljen bio pretreniran. Međutim, sa pomeranjem granice odlučivanja vidimo da je model znatno elastičniji na promene u njemu, pa smo tako uspeli da recall na testu značajno povećamo. Zato ćemo izabrati KNN za naš najbolji model, te ćemo pokušati da podešavanjem parametara sprečimo overfitting. Optimizovaćemo recall.

In [25]:
k_options = list(range(2,100))
weight_options = ["uniform", "distance"]
algorithm = ['auto','ball_tree','kd_tree']

param_grid = dict(n_neighbors=k_options, weights=weight_options, algorithm=algorithm)

print(f"Count of different values of params: {len(k_options)+len(weight_options)+len(algorithm)}")
print(f"Count of different model to be trained: {len(k_options)*len(weight_options)*len(algorithm)}")

Count of different values of params: 103
Count of different model to be trained: 588


In [26]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv = 10, scoring = 'recall', n_jobs=-1)
grid.fit(X_train, y_train)

/Users/dimitrijemilenkovic/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree'],
                         'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                         14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                                         24, 25, 26, 27, 28, 29, 30, 31, ...],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='recall', verbose=0)

In [27]:
grid.best_params_

{'algorithm': 'auto', 'n_neighbors': 2, 'weights': 'distance'}

In [28]:
grid.best_score_

0.6195668315049151

In [29]:
model_knn_best = grid.best_estimator_

In [30]:
evaluate(models+[model_knn_best], scores, X_test, y_test)

+------------------------+--------------------+--------------------+---------------------+--------------------+
| Model\Mera evaluacije  |   accuracy_score   |  precision_score   |     recall_score    |   roc_auc_score    |
+------------------------+--------------------+--------------------+---------------------+--------------------+
|   LogisticRegression   | 0.7868245743893413 | 0.5818181818181818 | 0.20915032679738563 | 0.5825655940207024 |
| DecisionTreeClassifier | 0.8179126572908957 |        0.6         |  0.5882352941176471 | 0.7367013791162398 |
|  KNeighborsClassifier  | 0.8164322723908216 | 0.6141732283464567 |  0.5098039215686274 | 0.7080120086312037 |
| RandomForestClassifier | 0.8638045891931903 | 0.7877358490566038 |  0.545751633986928  | 0.751344716514995  |
|  KNeighborsClassifier  | 0.8371576609918578 | 0.6319018404907976 |  0.673202614379085  | 0.7791850392469587 |
+------------------------+--------------------+--------------------+---------------------+--------------

Optimizovanjem knn meta-parametara dobili smo veoma dobar model. Zanimljivo je da ovaj model osim što postiže najbolji recall (u cilju čega je optimizovan), ima najbolji i roc_auc_score.

### Kreiranje ansambl modela

3. Napraviti 2 razlicita ansambl modela, bazirana na modelima iz koraka 2
    - optimizovati parametre ansambl modela
    - uporediti kvalitet ansambl i osnovnih modela

#### Stacking algoritam

In [31]:
from mlxtend.classifier import StackingClassifier

model_tree = DecisionTreeClassifier()
model_knn = KNeighborsClassifier(algorithm='auto', n_neighbors=2, weights='distance')

model_stacking = StackingClassifier(classifiers=[model_tree, model_knn], meta_classifier = DecisionTreeClassifier())

cv_scores = cross_val_score(model_tree, X_train, y_train, cv=10, scoring='recall')
print("Tree: " + str(cv_scores.mean()))

cv_scores = cross_val_score(model_knn, X_train, y_train, cv=10, scoring='recall')
print("KNN: " + str(cv_scores.mean()))

cv_scores = cross_val_score(model_stacking, X_train, y_train, cv=10, scoring='recall')
print("Stacking: " + str(cv_scores.mean()))

Tree: 0.5860524256651016
KNN: 0.6195226917057903
Stacking: 0.587539123630673


Stacking tehnika nam daje ok rezultate, ali značajno komplikuje model, tako da bismo je ipak preskočili pri ovakvim rezultatima.

#### Bagging algoritam

Već smo koristili jedan od algoritama iz ove klase. RandomForest je bagging algoritam koji radi uzorkovanje sa ponavljanjem slučaja i atributa. Probaćemo da napravimo još jedan bagging algoritam koji bi u osnovi koristio DT.

In [32]:
from sklearn.ensemble import BaggingClassifier

model_tree = DecisionTreeClassifier()
model = BaggingClassifier(base_estimator=model_tree, bootstrap=30)

cv_scores = cross_val_score(model, X_train, y_train, cv=10, scoring='recall')
cv_scores.mean()

0.5173904538341157

In [33]:
bagging_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(), bootstrap=30)
bagging_model.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort=False,
                                                        random_state=None,
                                                        splitter='best'),
    

In [34]:
scores = [accuracy_score, precision_score, recall_score, roc_auc_score]
e_models = [model_knn_best, bagging_model, model_rff]
evaluate(e_models, scores, X_train, y_train)

+------------------------+--------------------+--------------------+--------------------+--------------------+
| Model\Mera evaluacije  |   accuracy_score   |  precision_score   |    recall_score    |   roc_auc_score    |
+------------------------+--------------------+--------------------+--------------------+--------------------+
|  KNeighborsClassifier  |        1.0         |        1.0         |        1.0         |        1.0         |
|   BaggingClassifier    | 0.9863535385591876 | 0.9970414201183432 | 0.9426573426573427 | 0.9709181622974725 |
| RandomForestClassifier | 0.9879403364011425 | 0.9985272459499264 | 0.9482517482517483 | 0.9739206196102748 |
+------------------------+--------------------+--------------------+--------------------+--------------------+




In [35]:
evaluate(e_models, scores, X_test, y_test)

+------------------------+--------------------+--------------------+--------------------+--------------------+
| Model\Mera evaluacije  |   accuracy_score   |  precision_score   |    recall_score    |   roc_auc_score    |
+------------------------+--------------------+--------------------+--------------------+--------------------+
|  KNeighborsClassifier  | 0.8371576609918578 | 0.6319018404907976 | 0.673202614379085  | 0.7791850392469587 |
|   BaggingClassifier    | 0.8578830495928942 |        0.75        | 0.5588235294117647 | 0.7521390374331551 |
| RandomForestClassifier | 0.8638045891931903 | 0.7877358490566038 | 0.545751633986928  | 0.751344716514995  |
+------------------------+--------------------+--------------------+--------------------+--------------------+




Iako pokazuju neke rezultate, izgrađeni ansambl algoritmi su i dalje gori od optimizovanog KNN modela. Iz ugla tačnosti oni su dobri, međutim jako se loše bore sa našim nebalansiranim podacima, pa je recall i ne baš zadovoljavajuć.

### Balansiranje podataka

In [36]:
df.top_quality.value_counts(normalize=True)

0    0.783585
1    0.216415
Name: top_quality, dtype: float64

Kako što je već pomenuto, podaci su jako nebalansirani. Zaključili smo da nam je bitnija pozitivna klasa, koje u setu ima 5 puta manje nego negativne klase. S obzirom na to, modeli koje pravimo jako teško postižu dobar recall. 
Probaćemo da balansiramo trening skup kako bismo proverili da li ćemo tako dobiti bolje rezultate. Odlučeno je da se primeni undersampling tehnika - smanjićemo broj obzervacija negativne klase u trening skupu.

In [37]:
from imblearn.under_sampling import RandomUnderSampler

under_sampler = RandomUnderSampler(random_state=420)
X_train_b, y_train_b = under_sampler.fit_resample(X_train, y_train)

In [38]:
# optimized knn
model_knn_b = KNeighborsClassifier(algorithm='auto', n_neighbors=2, weights='distance')
model_knn_b.fit(X_train_b, y_train_b)

# bagging
bagging_model_b = BaggingClassifier(base_estimator=DecisionTreeClassifier(), bootstrap=30)
bagging_model_b.fit(X_train_b, y_train_b)

# rf
model_rf_b = RandomForestClassifier()
model_rf_b.fit(X_train_b, y_train_b)

b_models = [model_knn_b, bagging_model_b, model_rf_b]

In [39]:
evaluate(e_models + b_models, scores, X_test, y_test)

+------------------------+--------------------+--------------------+--------------------+--------------------+
| Model\Mera evaluacije  |   accuracy_score   |  precision_score   |    recall_score    |   roc_auc_score    |
+------------------------+--------------------+--------------------+--------------------+--------------------+
|  KNeighborsClassifier  | 0.8371576609918578 | 0.6319018404907976 | 0.673202614379085  | 0.7791850392469587 |
|   BaggingClassifier    | 0.8578830495928942 |        0.75        | 0.5588235294117647 | 0.7521390374331551 |
| RandomForestClassifier | 0.8638045891931903 | 0.7877358490566038 | 0.545751633986928  | 0.751344716514995  |
|  KNeighborsClassifier  | 0.7401924500370096 | 0.4582560296846011 | 0.8071895424836601 | 0.763881852581543  |
|   BaggingClassifier    | 0.7823834196891192 | 0.5131004366812227 | 0.7679738562091504 | 0.7772883635112738 |
| RandomForestClassifier | 0.8053293856402665 | 0.550351288056206  | 0.7679738562091504 | 0.7921208993964411 |
+

Modeli izgrađeni nad balansiranim podacima (poslednja 3 reda tabele) dali su značajno bolje rezultate. Uspeli smo da povećamo recall za čak 13% u poređenju sa prethodno najbolji modelom. 